In [5]:
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.callbacks import Callback
import pandas as pd
import numpy as np
import cv2
from PIL import Image
import keras
import subprocess
import os

import wandb
from wandb.keras import WandbCallback

In [82]:
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator

In [6]:
def load_fer2013():
    if not os.path.exists("fer2013"):
        print("Downloading the face emotion dataset...")
        subprocess.check_output("curl -SL https://www.dropbox.com/s/opuvvdv3uligypx/fer2013.tar | tar xz", shell=True)
    data = pd.read_csv("fer2013/fer2013.csv")
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = np.asarray(pixel_sequence.split(' '), dtype=np.uint8).reshape(width, height)
        face = cv2.resize(face.astype('uint8'), (width, height))
        faces.append(face.astype('float32'))

    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).as_matrix()

    val_faces = faces[int(len(faces) * 0.8):]
    val_emotions = emotions[int(len(faces) * 0.8):]
    train_faces = faces[:int(len(faces) * 0.8)]
    train_emotions = emotions[:int(len(faces) * 0.8)]
    
    return train_faces, train_emotions, val_faces, val_emotions


In [25]:
# loading dataset

train_faces, train_emotions, val_faces, val_emotions = load_fer2013()
num_samples, num_classes = train_emotions.shape

train_faces /= 255.
val_faces /= 255.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


In [22]:
train_faces.shape

(28709, 48, 48, 1)

In [122]:
datagen = ImageDataGenerator(
        #rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        #rescale=1./255,
        #shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
datagen.fit(train_faces, augment=True, rounds=8)

In [124]:
input_shape = (48, 48, 1)

config = run.config

#config.first_layer_convs = 32
#config.first_layer_conv_width = 3
#config.first_layer_conv_height = 3
#config.dropout = 0.2
#config.dense_layer_size = 128
#config.img_width = 28
#config.img_height = 28
config.num_epochs = 32

In [125]:
run = wandb.init()

W&B Run: https://app.wandb.ai/qualcomm/emotion-aug21/runs/5cou83o5
Wrap your training loop with `with wandb.monitor():` to display live results.


In [ ]:
model = Sequential()

model.add(Conv2D(32, (3,3), 
    #(config.first_layer_conv_width, config.first_layer_conv_height),
    input_shape=(48, 48,1),
    padding='valid',
    activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3,3), 
    padding='valid',
    activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), 
    padding='valid',
    activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(4*64*7, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation="softmax"))

model.compile(optimizer='adam', loss='categorical_crossentropy',
metrics=['accuracy'])

#model.load_weights("emotion.h5")

#model.fit(train_faces, train_emotions, batch_size=32,
#        epochs=config.num_epochs, verbose=1, callbacks=[
#            WandbCallback(data_type="image", labels=["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"])
#        ], validation_data=(val_faces, val_emotions))


train_generator = datagen.flow(train_faces, train_emotions, batch_size=32)
val_generator = datagen.flow(val_faces, val_emotions, batch_size=32)
model.fit_generator(train_generator,
            steps_per_epoch=len(train_faces)//32, 
            epochs=config.num_epochs, verbose=1,
            #callbacks=[
            #    WandbCallback(data_type="image", labels=["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"])
            #],
            validation_data=val_generator,
            validation_steps=len(val_faces)//32)


#for e in range(config.num_epochs):
#    print("Epoch", e)
#    batches = 0
#    for (x_batch, y_batch), (val_x, val_y) in zip(
#                    datagen.flow(train_faces, train_emotions, batch_size=32),
#                    datagen.flow(val_faces, val_emotions, batch_size=32)):
#        model.fit(x_batch, y_batch, validation_data=(val_x, val_y), verbose=0)
#        batches += 1
#        if batches >= len(train_faces) / 32:
#            break
#    model.evaluate(val_faces, val_emotions)
            
    

Epoch 1/32
886/897 [============================>.] - ETA: 0s - loss: 1.8042 - acc: 0.2489

In [112]:
len(train_faces)//32

897

In [15]:
model.save("emotion.h5")